In [3]:
!pip install -U albumentations
!pip install opencv-python
!pip install torchvision
!pip install imgaug

     ---------------------------------------- 0.0/948.0 kB ? eta -:--:--
     ---------------------------------------- 10.2/948.0 kB ? eta -:--:--
     --- ----------------------------------- 92.2/948.0 kB 1.1 MB/s eta 0:00:01
     ----------------------------------- -- 890.9/948.0 kB 7.0 MB/s eta 0:00:01
     -------------------------------------- 948.0/948.0 kB 6.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 1.4/1.4 MB 46.0 MB/s eta 0:00:00


In [6]:
import os
import cv2
import numpy as np
import imgaug as ia
from imgaug import augmenters as iaa
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage

# Set the paths to your images and labels
images_path = "./sea-turtles-1-test/all/images/"
labels_path = "../sea-turtles-1-test/all/labels/"

# List all image files in the images_path directory
image_files = [f for f in os.listdir(images_path) if f.endswith('.jpg')]

# Define the augmentation sequence
seq = iaa.Sequential([
    iaa.Fliplr(0.5),        # horizontal flips
    iaa.Affine(rotate=(-45, 45)),  # random rotations
    iaa.GaussianBlur(sigma=(0, 3.0))  # random Gaussian blur
])

# Iterate through each image file
for image_file in image_files:
    # Load the image
    image_path = os.path.join(images_path, image_file)
    image = cv2.imread(image_path)
    
    # Load the corresponding label file
    label_path = os.path.join(labels_path, image_file.replace('.jpg', '.txt'))
    with open(label_path, 'r') as file:
        # Read bounding box coordinates from the label file
        bounding_boxes = []
        for line in file:
            values = [float(val) for val in line.strip().split()]
            bounding_box = BoundingBox(
                x1=values[1],
                y1=values[2],
                x2=values[1] + values[3],
                y2=values[2] + values[4]
            )
            bounding_boxes.append(bounding_box)

        # Apply the augmentation to the image and bounding boxes
        augmented_image, augmented_bounding_boxes = seq(image=image, bounding_boxes=bounding_boxes)

        # Display the augmented image with bounding boxes (for visualization purposes)
        image_with_boxes = augmented_image.copy()
        for box in augmented_bounding_boxes.bounding_boxes:
            cv2.rectangle(image_with_boxes, (int(box.x1), int(box.y1)), (int(box.x2), int(box.y2)), (0, 255, 0), 2)

        cv2.imshow("Augmented Image", image_with_boxes)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

        # Save the augmented image and updated labels (if needed)
        augmented_image_path = os.path.join(images_path, f"aug_{image_file}")
        cv2.imwrite(augmented_image_path, augmented_image)

        augmented_label_path = os.path.join(labels_path, f"aug_{image_file.replace('.jpg', '.txt')}")
        with open(augmented_label_path, 'w') as output_file:
            for box in augmented_bounding_boxes.bounding_boxes:
                output_file.write(f"0 {box.x1} {box.y1} {box.width} {box.height}\n")


error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1272: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'
